In [18]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd

load_dotenv()

apiKey = os.environ.get('API_KEY')
print(apiKey)

key


In [57]:
## Endpoints importants 
#
# Acccount V1
# Summoner V4
# League   V4
# Match    V5 

def getPUUIDByRiotId(tagLine, gameName, apiKey, region='europe'):

	lien = f'https://{region}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={apiKey}'
	retour = requests.get(lien)
	if (retour.status_code == 200):
		return retour.json()['puuid']
	else:
		print("Erreur lors de la récupération des données du compte Riot ID")
		print(f"Code d'erreur: {retour.status_code}")
		print(f"Message d'erreur: {retour.text}")
		return None
	
def getIdByRiotPUUID(puuid, apiKey, region='europe'):

	lien = f'https://{region}.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={apiKey}'
	retour = requests.get(lien)
	if (retour.status_code == 200):
		gameName = retour.json()['gameName']
		tagLine = retour.json()['tagLine']

		return f'{gameName}#{tagLine}'
	else:
		print("Erreur lors de la récupération des données depuis le PUUID")
		print(f"Code d'erreur: {retour.status_code}")
		print(f"Message d'erreur: {retour.text}")
		return None
	

def getMatchHistory(puuid, apiKey, region='europe', start=0, count=20):

	lien = f'https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}&api_key={apiKey}'
	retour = requests.get(lien)
	if (retour.status_code == 200):
		return retour.json()
	else:
		print("Erreur lors de la récupération de l'historique des matchs")
		print(f"Code d'erreur: {retour.status_code}")
		print(f"Message d'erreur: {retour.text}")
		return None
	
def getDataFromMatchId(matchId, apiKey, region='europe'):

	lien = f'https://{region}.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={apiKey}'
	retour = requests.get(lien)
	if (retour.status_code == 200):
		return retour.json()
	else:
		print("Erreur lors de la récupération des données du match")
		print(f"Code d'erreur: {retour.status_code}")
		print(f"Message d'erreur: {retour.text}")
		return None
	

# faire un tableau avec le kda de chaque joueur dans le dernier match
# remplacer le summonerName par le nom du joueur dans le match
def getKDAFromMatch(matchData):
	players = matchData['info']['participants']
	kda_list = []
	cpt = 0
	for player in players:
		summonerName = getIdByRiotPUUID(matchData['metadata']['participants'][cpt], apiKey=apiKey)
		kills = player['kills']
		deaths = player['deaths']
		assists = player['assists']
		kda = f"{kills}/{deaths}/{assists}"
		kda_list.append({'summonerName': summonerName, 'KDA': kda})
		cpt+= 1
	
	return pd.DataFrame(kda_list)

In [28]:
myPuuid = getPUUIDByRiotId(gameName='MelècheLAbricot', tagLine='juter', apiKey=apiKey)
getIdByRiotPUUID(myPuuid, apiKey)

'MelècheLAbricot#juter'

In [ ]:
matchs     = getMatchHistory(myPuuid, apiKey)
match0Data = getDataFromMatchId(matchs[0], apiKey)
'''
print (match0Data)
print (match0Data['info']['participants'][0]['summonerName'])
print (match0Data['metadata']['participants'][0])
print (getIdByRiotPUUID((match0Data['metadata']['participants'][0]), apiKey=apiKey))
print()'''
getKDAFromMatch(match0Data)

#Faire une moyenne des kda d'un joueur sur les 20 derniers matchs
#Faire les wins / loses sur les 20 derniers matchs
#Faire le perso avec le plus de wins sur les ... derniers matchs
#Faire le perso avec le plus de kills sur les ... derniers matchs
#Faire le perso avec le plus de deaths sur les ... derniers matchs


,summonerName,KDA
0,zialde#3837,17/7/6
1,Ironveko#EUW,7/9/15
2,Keksツツ#1010,12/9/10
3,Velo999#EUW,7/11/14
4,Redo of Healer#EUWso,7/11/18
5,MelècheLAbricot#juter,8/16/13
6,Vaswolf EUW#Howl,13/10/12
7,CaIłum#EUW,16/4/8
8,BIG Adryz#BIG,8/15/9
9,Micro icë#EUW,2/5/30
